In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

# Uncovering World Events using Twitter Hashtags

## ... and learning about Spark `DataFrames` along the way

In this notebook, we will use temporal information about Twitter hashtags to discover trending topics and potentially uncover world events as they occurred. You've already used this dataset in a previous homework, but here we will be working with it in Apache Spark.

### Hashtags

The idea here is that when an event is happening and people are having a conversation about it on Twitter, a set of uniform hashtags that represent the event spontaneously evolves. Twitter users then use those hashtags to communicate with one another. Some hashtags, like `#RT` for "retweet" or just `#retweet` are used frequently and don't tell us much about what is going on. But a sudden appearance of a hashtag like `#oscars` probably indicates that the oscars are underway. For a particularly cool example of this type of analysis, check out [this blog post about earthquake detection using Twitter data](https://blog.twitter.com/official/en_us/a/2015/usgs-twitter-data-earthquake-detection.html) (although they search the text and not necessarily hashtags).

## Initialize the `SparkSession`

In [ ]:
import getpass
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('twitter-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '4g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

In [ ]:
spark = SparkSession(sc)

## Set up the hashtag `DataFrame` (10 points / 60)

We have prepared the hashtag data spanning the time from April-July 2016. This is a significant time in modern European history, e.g. see [Brexit](https://en.wikipedia.org/wiki/Brexit). Lets see if we can see any interesting trends about these events in the Twitter data. 

In [ ]:
from datetime import datetime

import pyspark.sql.functions as functions

### TODO (1 point/10)

Load the **parquet** data from `/datasets/twitter_hashtags` into a Spark dataframe using the appropriate `SparkSession` method. 

Look at the first few rows of the dataset - note the timestamp and its units!

In [ ]:
df = spark.read.<FILL IN>

In [ ]:
df.show()

### User-defined functions

A neat trick of spark dataframes is that you can essentially use something very much like an RDD `map` method but without switching to the RDD. If you are familiar with database languages, this works very much like e.g. a user-defined function in SQL. 

So, for example, if we wanted to make a user-defined python function that returns the hashtags in lowercase, we could do something like this:

In [ ]:
@functions.udf
def lowercase(text):
    """Convert text to lowercase"""
    return text.lower()

The @functions.udf is a "decorator" -- this is really handy python syntactic sugar and in this case is equivalent to:

```
def lowercase(text):
    return text.lower()
    
lowercase = functions.udf(lowercase)
```

It basically takes our function and adds to its functionality. In this case, it registers our function as a pyspark dataframe user-defined function (UDF).

Using these UDFs is very straightforward and analogous to other Spark dataframe operations. For example:

In [ ]:
df.select(lowercase(df.hashtag)).show()

Using a framework like Spark is all about understanding the ins and outs of how it functions and knowing what it offers. One of the cool things about the dataframe API is that many functions are already defined for you (turning strings into lowercase being one of them). 

### TODO (2 points / 10)

Find the Spark python API documentation. Look for the `sql` section and find the listing of `sql.functions`. Repeat the above (turning hashtags into lowercase) but use the built-in function.

In [ ]:
df.<FILL IN>

We'll work with a combination of these built-in functions and user-defined functions for the remainder of this homework. 

### TODO (7 points/10)

Create `english_df` consisting of **lowercase** hashtags from only english-language tweets. In addition, convert the timestamp to a more readable format like this and name the column `date`:

```
2016-04-01 08:41:00
```

Your `english_df` should look something like this:

```
+-------------+----+-----------+-------------------+
| timestamp_ms|lang|    hashtag|               date|
+-------------+----+-----------+-------------------+
|1459492860658|  en|        win|2016-04-01 08:41:00|
|1459492860658|  en|competition|2016-04-01 08:41:00|
|1459492860658|  en|   giveaway|2016-04-01 08:41:00|
|1459492860665|  en|    5sosfam|2016-04-01 08:41:00|
|1459492860665|  en|bestfanarmy|2016-04-01 08:41:00|
+-------------+----+-----------+-------------------+
```

## Twitter hashtag trends (50 points / 60)

In this section we will try to do a slightly more complicated analysis of the tweets. Our goal is to get an idea of tweet frequency as a function of time for certain hashtags. 

Lets build this up in steps. First, lets see how we can start to organize the tweets by their timestamps. 

As a first easy example, lets say we just want to count the number of tweets per minute over the entire span of our data. For this, we first need a "global" minute value, e.g. "minute of the year" or something similar. 

Spark provides us with some handy built-in dataframe functions that are made for transforming date and time fields. 

Have a look [here](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions) to see the whole list of custom dataframe functions - you will need to use them to complete the next set of TODO items.

Note that the functions can be combined. Consider the following dataframe and its transformation:

In [ ]:
from pyspark.sql import Row

# create a sample dataframe with one column "degrees" going from 0 to 180
test_df = spark.createDataFrame(sc.range(180).map(lambda x: Row(degrees=x)), ['degrees'])

# define a function "sin_rad" that first converts degrees to radians and then takes the sine using built-in functions
sin_rad = functions.sin(functions.radians(test_df.degrees))

# show the result
test_df.select(sin_rad).show()

### DataFrames `groupBy`

We used `groupBy` already in the previous notebook, but here we will take more advantage of its features. 

One important thing to note is that unlike other RDD or DataFrame transformations, the `groupBy` does not return another DataFrame, but a `GroupedData` object instead, with its own methods. These methods allow you to do various transformations and aggregations on the data of the grouped rows. 

Conceptually the procedure is a lot like this:

![groupby](https://i.stack.imgur.com/sgCn1.jpg)

The column that is used for the `groupBy` is the `key` - once we have the values of a particular key all together, we can use various aggregation functions on them to generate a transformed dataset. In this example, the aggregation function is a simple `sum`. In the simple procedure below, the `key` will be the hashtag.

### TODO (1 point / 50)

Calculate the top five most common hashtags in the whole english-language dataset.

This should be your result:

```
+-------------+------+
|      hashtag| count|
+-------------+------+
|   mtvhottest|800829|
|veranomtv2016|539028|
| iheartawards|447651|
|  bestfanarmy|429739|
|   teenchoice|345250|
+-------------+------+
```

In [ ]:
english_df.<FILL IN>

## Daily hashtag trends

Now we will start to complicate the analysis a bit. Remember, our goal is to uncover trending topics on a timescale of a few days. A much needed column then is simply `day`. To convert the date string into day-of-year, you can use the built-in [dayofyear](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.dayofyear) function. 

In the subsequent sections we will then not only see which hashtags are globally most popular, but which ones experience the biggest changes in popularity - those are the "trending" topics. If there is suddenly a substantial increase of a hashtag over a matter of a day or two, it may signify an event taking place. 

### TODO (2 points / 50)

Create a dataframe called `daily_hashtag` that includes the columns `month`, `week`, `day` and `hashtag`. Use the `english_df` you made above to start, and make sure you find the appropriate spark dataframe functions to make your life easier. Show the result.

Try to match this view:

```
+-----+----+---+-----------+
|month|week|day|    hashtag|
+-----+----+---+-----------+
|    4|  13| 92|        win|
|    4|  13| 92|competition|
|    4|  13| 92|   giveaway|
|    4|  13| 92|    5sosfam|
|    4|  13| 92|bestfanarmy|
+-----+----+---+-----------+
```

### TODO (2 points / 50)

Now we want to calculate the number of times a hashtag is used per day. Sort in descending order of daily counts and show the result. Call the resulting dataframe `day_counts`.

Your output should look like this:

```
+---+------------+----+------+
|day|     hashtag|week| count|
+---+------------+----+------+
| 94|iheartawards|  13|190884|
| 94| bestfanarmy|  13|185789|
| 93|iheartawards|  13|124645|
| 93| bestfanarmy|  13|122741|
| 95|iheartawards|  14| 91921|
+---+------------+----+------+
```

<div class="alert alert-info">
<p>Make sure you use `cache()` when you create `day_counts` because we will need it in the steps that follow!</p>
</div>

### TODO (5 points / 50)

To get an idea of the most popular persistent hashtags, use the `week` column that we kept around to calculate the average number of times per day that a hashtag occurs in a week. Sort in descending order and show the top 20.

In [ ]:
day_counts.<FILL IN>

### Using `Window` functions 

Window functions are another awesome feature of dataframes. They allow users to accomplish complex tasks using very concise and simple code. 

Above we computed just the hashtag that had the most occurrences on *any* day. Now lets say we want to know the top tweets for *each* day.  

This is a non-trivial thing to compute and requires "windowing" our data. I recommend reading this [window functions article](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html) to get acquainted with the idea. You can think of a window function as a fine-grained and more flexible `groupBy`. 

There are two things we need to define to use window functions:

1. the "window" to use, based on which columns (partitioning) and how the rows should be ordered 
2. the computation to carry out for each windowed group, e.g. a max, an average etc.

Lets see how this works by example. We will define a window function, `daily_window` that will partition data based on the `day` column. Within each window, the rows will be ordered by the daily hashtag count that we computed above. Finally, we will use the rank function **over** this window to give us the ranking of top tweets. 

In the end, this is a fairly complicated operation achieved in just a few lines of code! (can you think of how to do this with an RDD??)

In [ ]:
from pyspark.sql import Window

First, we specify the window function and the ordering:

In [ ]:
daily_window = Window.partitionBy('day').orderBy(functions.desc('count'))

The above window function says that we should window the data on the `day` column and order it by count. 

Now we need to define what we want to compute on the windowed data. We will start by just calculating the daily ranking of hashtags, so we can use the helpful built-in `functions.rank()` and sort:

In [ ]:
daily_rank = functions.rank() \
                      .over(daily_window) \
                      .alias('rank')

Now we have the pieces we need to compute the top five hashtags for each day in our data:

In [ ]:
(day_counts.select('day', 'count', 'hashtag', daily_rank)
           .filter('rank <= 5')
           .sort('day', 'rank')
           .show(20))

### A more complicated `Window` example: calculating the daily standard deviation based on a weekly window

The above list of top daily hashtags is interesting, but it really only uncovers very frequent hashtags. We want to try and identify "world events" and for that we need to detect trending hashtags by spotting a rapid *change* in the tweeting pattern. 

One very simple model to do this is to just look at the whether the hashtags mentions on a given day are much higher than on average. For this, we can calculate the daily standard deviation using a baseline mean, such as a week.

Remember, variance is just:

$$\sigma^2 = \sum_i{(x_i-\mu)^2},$$

where $x_i$ is the daily count of a hashtag and $\mu$ is the weekly average. 

So the procedure is something like this: 

1. calculate $\mu$ for each hastag and each week
2. compute the sum of the differences squared 
3. order tweets for each week in order of decreasing $\sigma$


## TODO - Compute the top five trending tweets per week (25 points / 50)

Use window functions to calculate the variance for each hashtag and each week. Produce a weekly ranking of tweets based on their variance and print the top five tweets for each week. The output should look like this:

```
Top ten hashtags with highest variation each week
+----+--------------------+--------------------+----+
|week|             hashtag|            variance|rank|
+----+--------------------+--------------------+----+
|  13|        iheartawards|1.203149592866666...|   1|
|  13|         bestfanarmy|1.137100003266666...|   2|
|  13|             5sosfam|1.4885764806666665E9|   3|
|  13|        directioners|1.3716957286666665E9|   4|
|  13|         harmonizers|        9.27531104E8|   5|
|  13|    aldub3rdebawards|         1.8595976E7|   6|
|  13|      memeablemoment|1.2742160666666668E7|   7|
|  13|          aprilfools|1.0030552666666668E7|   8|
|  13|      jfcshawnmendes|   9252520.666666666|   9|
|  14|        iheartawards| 7.183277916857142E9|   1|
|  14|         bestfanarmy| 5.907355471714287E9|   2|
|  14|             5sosfam|1.4584854168571427E9|   3|
|  14|        wrestlemania|2.9820363142857146E8|   4|
|  14|         harmonizers|2.9375777371428573E8|   5|
|  14|        directioners|1.9487411342857143E8|   6|
|  14|         sofantastic| 3.819015571428572E7|   7|
|  14|nationalchampionship|3.7476486857142866E7|   8|
|  14|     dolceamoresayit|         3.0816227E7|   9|
|  15|   aldub9thmonthsary| 8.463613606666669E8|   1|
|  15|            mambaday|1.2468111571428573E8|   2|
|  15|  dolceamoreiloveyou|1.0015080520000002E8|   3|
|  15|           demdebate| 6.480662342857144E7|   4|
|  15|         sofantastic| 6.194139885714285E7|   5|
|  15|        thankyoukobe|3.3031526857142862E7|   6|
|  15|         harmonizers|2.2504020857142862E7|   7|
|  15|   aldub39thweeksary|        2.07642592E7|   8|
|  15|     aldubsummerlove|        2.05949015E7|   9|
|  16|  letsplay1dreamboy2| 4.318556866666666E7|   1|
|  16|            lemonade| 4.086382571428571E7|   2|
|  16|            earthday|3.8029611428571425E7|   3|
|  16|   aldub40thweeksary|        3.45427468E7|   4|
|  16|              prince|         3.4445144E7|   5|
|  16|pilipinasdebates2016|3.3852825428571425E7|   6|
|  16|      aldubgiftofjoy|3.1067077333333325E7|   7|
|  16|dolceamorethebigm...|       2.880975475E7|   8|
|  16|      aldubacting101|        2.50469215E7|   9|
|  17|   dolceamorevenezia|         7.9941618E7|   1|
|  17|               views| 6.452821571428571E7|   2|
|  17|   aldub41stweeksary| 2.975588719999999E7|   3|
|  17|                whcd|         2.5021998E7|   4|
|  17|       gameofthrones|         2.0947352E7|   5|
|  17|      aldubactoflove|1.9896217333333332E7|   6|
|  17|   aldubgettingready|1.7553324833333332E7|   7|
|  17|  aldubangpagtatapat|        1.63448912E7|   8|
|  17|     aldubhbdbossing|         1.4679638E7|   9|
|  18|             metgala| 3.608393877142856E8|   1|
|  18|   dolceamorethesong|         6.2369341E7|   2|
|  18| 화양연화onstageepilogue| 4.833936333333333E7|   3|
|  18|  maythe4thbewithyou|2.5003587714285716E7|   4|
|  18|          mothersday|2.4432857428571425E7|   5|
|  18|      aldubbonvoyage|2.2385640666666664E7|   6|
|  18|   aldub42ndweeksary|         2.1608336E7|   7|
|  18|    aldubfinalscript|2.0915212833333332E7|   8|
|  18|  aldubtickettoitaly|1.9933257333333332E7|   9|
|  19|          teenchoice|1.4985840377142856E9|   1|
|  19|          eurovision|2.6509957742857146E8|   2|
|  19|dolceamoreitsallc...|      1.0733945675E8|   3|
|  19| dolceamoreforgotten|          6.781391E7|   4|
|  19| dolceamoremeanttobe| 6.028717319999999E7|   5|
|  19|               방탄소년단| 3.573195485714285E7|   6|
|  19|                 bts| 3.478692685714286E7|   7|
|  19|choicemusicgroupmale|2.6838739333333336E7|   8|
|  19|   aldub43rdweeksary|        2.10542772E7|   9|
|  20|     1dreamboy2today| 9.333881666666669E7|   1|
|  20|  aldub10thmonthsary|4.0754155428571425E7|   2|
|  20|   aldub44thweeksary|        2.28811148E7|   3|
|  20|  aldubloversinitaly|2.1738495714285716E7|   4|
|  20|   aldubhappyinitaly|1.9249258833333332E7|   5|
|  20| dolceamorewhitelies|1.6688452833333336E7|   6|
|  20|      aldubwemissyou|         1.4809568E7|   7|
|  20|     aldubtiamoitaly|         1.4410507E7|   8|
|  20|      voicesavelaith|1.1729224666666666E7|   9|
|  21|               bbmas| 3.598061197142857E8|   1|
|  21| dolceamorepanaginip|         4.4339942E7|   2|
|  21|      dolceamoretuko|         3.0670345E7|   3|
|  21|            uclfinal| 3.018978742857142E7|   4|
|  21|       1dreamboy2now|2.6601552833333336E7|   5|
|  21|    aldubtouchoflove| 2.164150685714286E7|   6|
|  21|dolceamoresatotoo...|       2.051140475E7|   7|
|  21|   aldub45thweeksary|          2.037315E7|   8|
|  21|        aldubsoclose|2.0267521333333336E7|   9|
|  22|   dolceamorebistado|         2.5340328E7|   1|
|  22|    aldubmissinghalf|1.7461188666666668E7|   2|
|  22|     aldubhomeatlast|         1.3510819E7|   3|
|  22|      aldubbackatone|   9672338.666666668|   4|
|  22|         memorialday|           9546990.0|   5|
|  22|     memorialday2016|           4911206.0|   6|
|  22|                 raw|          2747458.75|   7|
|  22|          teenchoice|           2385046.0|   8|
|  22|               5h727|          2123164.75|   9|
|  23|       mtvawardsstar| 2.808470288333333E8|   1|
|  23|        ripchristina|       1.260078125E8|   2|
|  23|             orlando| 1.177854708333333E8|   3|
|  23|            euro2016|         4.5896674E7|   4|
|  23|dolceamoredreamin...|        3.41199572E7|   5|
|  23|       dolceamoretlc|         3.3426619E7|   6|
|  23|   aldub47thweeksary|        2.00338128E7|   7|
|  23|dolceamoreclosetoyou|         1.8993046E7|   8|
|  23|       aldubtrumpets|         1.7667704E7|   9|
|  24|  aldub11thmonthsary|         1.0705437E8|   1|
|  24|             orlando|          6.740826E7|   2|
|  24|          loveislove| 4.129454283333334E7|   3|
|  24|          tonyawards| 2.533964083333333E7|   4|
|  24|          fathersday| 2.053040483333333E7|   5|
|  24|              e32016|1.7640861333333332E7|   6|
|  24|      aldubthesender|         1.6480344E7|   7|
|  24|     aldubforthelove|          1.634729E7|   8|
|  24|      alduballmylife|         1.5920202E7|   9|
|  25|              brexit| 5.619198574285715E8|   1|
|  25|           nbafinals| 2.034930414285714E8|   2|
|  25|               euref|1.1304269685714287E8|   3|
|  25|       veranomtv2016| 9.154586285714287E7|   4|
|  25|       nobillnobreak| 9.102413085714284E7|   5|
|  25|        eurefresults|       7.245406475E7|   6|
|  25|          teenchoice| 4.428511942857142E7|   7|
|  25|      dolceamorehhww|       4.015488875E7|   8|
|  25| 1dreamboy2flashback|        3.13236125E7|   9|
|  26|       veranomtv2016|3.4067891885714287E8|   1|
|  26|           betawards|3.2421500685714275E8|   2|
|  26|       gameofthrones| 5.786883971428572E7|   3|
|  26|        growingupshy| 4.110965285714285E7|   4|
|  26|            euro2016|3.0678978857142862E7|   5|
|  26|                 eng| 2.311730685714286E7|   6|
|  26|heterosexualprideday|2.2937988799999997E7|   7|
|  26|   dolceamorewarning|         2.1789481E7|   8|
|  26|   aldubiyamin14days|1.9558076833333332E7|   9|
|  27|       veranomtv2016| 6.172811897142859E8|   1|
|  27|       altonsterling|       4.353550708E8|   2|
|  27|              dallas|3.0474667485714287E8|   3|
|  27|          teenchoice|        1.44406374E8|   4|
|  27| throwback1dreamboy2|1.0612579466666664E8|   5|
|  27|    blacklivesmatter| 9.825445142857142E7|   6|
|  27|     philandocastile|         8.6897369E7|   7|
|  27|aldubmainecelebra...|4.7500080666666664E7|   8|
|  27|            euro2016| 3.652664371428572E7|   9|
|  28|          mtvhottest|3.0488708594285717E9|   1|
|  28| aldub1stanniversary|      4.1578049275E8|   2|
|  28|       veranomtv2016|1.2067342285714284E8|   3|
|  28|         prayfornice|         9.3551121E7|   4|
|  28|              turkey| 8.518514742857143E7|   5|
|  28|aldubimagineyouandme| 5.561218883333333E7|   6|
|  28|            euro2016|         2.9229292E7|   7|
|  28|                nice|2.7864511428571437E7|   8|
|  28|iyamthegrandpremiere|2.3359341333333332E7|   9|
|  29|          mtvhottest| 5.754052757142857E8|   1|
|  29|            rncincle|2.6088884942857146E8|   2|
|  29|          6yearsof1d| 1.102624657142857E8|   3|
|  29|       veranomtv2016| 9.774578085714285E7|   4|
|  29|kimexposedtaylorp...| 9.451060371428569E7|   5|
|  29|          teenchoice| 9.222880542857142E7|   6|
|  29|6yearsofonedirection| 6.410710720000001E7|   7|
|  29|dolceamorechangedman|         5.4128522E7|   8|
|  29|dolceamorefightin...| 5.309390866666666E7|   9|
|  30|          mtvhottest|         5.3450441E8|   1|
|  30|        demsinphilly| 4.432001374285714E8|   2|
|  30|     savemarinajoyce|        2.00240236E8|   3|
|  30|       veranomtv2016| 7.138472542857143E7|   4|
|  30|dolceamoreconnivance|5.7476010833333336E7|   5|
|  30|dolceamorelostand...|       4.581946275E7|   6|
|  30|  dolceamoresabotage|       4.321053875E7|   7|
|  30| dolceamorepatigasan| 3.897952866666666E7|   8|
|  30|    dolceamoreaminan|3.2220744799999993E7|   9|
+----+--------------------+--------------------+----+
```

You should be seeing some weekly lists here that are starting to make sense. Lots of these are related to viral twitter campaigns -- however, you should see the top trending hashtag of week 25 in 2016 to be "brexit" (closely followed by "nbafinals"). Others in the top ten include "euref" and "eurefresults". What other newsworthy events can you spot?

## Plotting the timeseries of hashtag counts 

### TODO (5 points / 50)

These standard deviation lists we obtained above are nice, but lets actually visualize some data. Plot a time series of a few of the more interesting hashtags from the weekly trending list above. Remember that you can filter down the dataframe and then use `toPandas` to bring it to the notebook for plotting.

### TODO: Use KMeans to cluster hashtags based on the daily count timeseries (10 points / 50)

Hint: 

1. start from the `daily_hashtag` dataframe you made earlier and use the `pivot` method of `GroupedData` to create a matrix of hashtags as rows and daily counts as columns. 
2. Use the `VectorAssembler` from the ML library to create the feature vector which will consist of daily counts. 

Train the model and calculate the cluster membership for all hashtags. Show the cluster that includes "brexit".

This last part is deliberately under-documented - be creative and see if you can get the clustering to give you meaningful hashtag groupings.